In [1]:
%pwd

'/Users/yangbo/Documents/projects/qamp_gse/test/ipynb/gse_vqe/backends/fake_jakarta'

# COMMON --- LIBRARIES AND UTILS

In [2]:
import sys, pickle, time, importlib
sys.path.append("../")
from pprint import pprint
import numpy as np
np.random.seed(42)
np.set_printoptions(threshold=sys.maxsize)
import scipy as spy
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("ggplot")

### Define Hamiltonian class

Here we are using our original `Hamiltonian` which can take its exponentiation.
This class can be replaced by `qiskit.opflow` in the future.

### Define the class and function for density matrix

Here we are extending the `qiskit.DensityMatrix` to `ExtendedDM` class, by adding matrix multiplication and matrix exponentiation.
For other functions, we defined
- `pauli_str_to_dm(pauli_str) -> ExtendedDM`
- `hamiltonian_to_dm(hamiltonian) -> ExtendedDM`
- `add_observable_to_dm(density_matrix: DensityMatrix, basis: str, dm_endian: str = "big_endian", basis_endian: str = "little_endian") -> ExtendedDM`
- `dm_to_hist(density_matrix: DensityMatrix, dm_endian: str = "big_endian", hist_endian: str = "big_endian") -> Counts`
- `dms_to_hists(dm_list: List[DensityMatrix],  dm_endian: str = "big_endian", hist_endian: str = "big_endian") -> List[Counts]`
- `get_density_matrices(result: Result) -> List[DensityMatrix]`

In [3]:
from utils import *

# COMMON --- DSP

Here we are defining the functions for dual-state purification.

### Define supporting functions for encoders of DSP
The function `update_observable_dsp` can output the DSP quantum circuit for a given observable.
To make it hardware-efficient, we have to feed the `connection_graph` which indicates how the qubits are physically connected.
The function `make_connection_order` will find a best way to apply the two qubit gates among qubits which are sparsely connected, while this is kept to be an identity function so far.

- `make_connection_order(z_index: int, connection_graph: List[List[int]])`
- `update_observable_dsp(observable: Union[QuantumCircuit, Gate, str], z_index: int, connection_graph: List[List[int]]) -> QuantumCircuit`

### Define encoder functions of DSP

The function `make_qc_dsp_ancilla` creates and outputs the DSP quantum circuit given a target state, its dual state, and an observable.
There are three options for adding the quantum circuit for dual state:
- if we feed `qc_v_inv`, it directly use the quantum cirucit `qc_v_inv` as the inverse circuit of dual state.
- if we feed `qc_v_dual` instead of `qc_v_inv`, it takes inverse operation 
- if we do not specify any of `qc_v_dual` or `qc_v_inv`, then this function will automatically take the inverse operation of `qc_u`.

Other Attributes
- `qc_name`: specifies whether the obsevable is identity or not.
- `measurement`: if `measurement == False`, the function will return the quantum circuit without measurement operations with computational basis.
- `z_index`: indicates which qubit is to be directly connected with the ancilla qubit.
- `connection_graph`: indicates how the physical qubits are connected.
- `barrier`: decides barriers are inserted among the quantum gates in the circutis.

```python
make_qc_dsp_ancilla(qc_u: QuantumCircuit = None, 
                    qc_v_dual: QuantumCircuit = None, 
                    qc_v_inv: QuantumCircuit = None,
                    observable: Union[QuantumCircuit, Gate, str] = None, 
                    qc_name: str = None, 
                    measurement: bool = True,
                    z_index: int = None,
                    connection_graph: List[List[int]] = None,
                    barrier: bool = False) -> QuantumCircuit
```

### Define supporting functions for decoders of DSP
These supporting functions reshape the measurement results into the format that is suitable for compute the expectation value by DSP.
In particular, `reshape_hist` is used in `compute_energy_gse_dsp_ancilla`.

- `make_conditioned_hist(hist: Dict[str, Union[int, float]], condition_state: str, qr_poses: List[int]) -> Dict[str, Union[int, float]]`
- `remove_space_from_hist_states(hist: Dict[str, Union[int, float]]) -> Dict[str, Union[int, float]]`
- `reshape_hist(hist: Dict[str, Union[int, float]]) -> List[Dict[str, Union[int, float]]]`

### Define decoder functions of DSP

These functions compute the trace of an observable and a quantum state according to DSP.
- `compute_trace_dsp_ancilla_numerator(n: int, hist: Dict[str, Union[int, float]]) -> float`
- `compute_trace_dsp_ancilla_denominator(n: int, hist: Dict[str, Union[int, float]]) -> float`

In [4]:
from dsp import *

# COMMON --- GSE

GSE computes the mitigated expectation value assuming the following ansatz.
$$\rho_{\mathrm{EM}}=\frac{P^{\dagger} A P}{\operatorname{Tr}\left[P^{\dagger} A P\right]}$$
where $$\sigma_i=\sum_k \beta_k^{(i)} \prod_{l=1}^{L_k} U_{l k}^{(i)} \rho_{l k}^{(i)} V_{l k}^{(i)}$$

### Define the function for the encoder of GSE

The function `make_qcs_gse_dsp_ancilla` outputs the quantum circuits for GSE, given the target Hamiltonian and the quantum circuits of subspaces.
So far, the subspaces are defined by `sigma_list` and `pauli_list`, which can only indicate restricted subspaces:
- fault subspaces such as $\{\rho(\epsilon), \rho(2\epsilon), \rho(3\epsilon)\}$
- power subspaces with two-dim GSE matrix such as $\{\rho, H\rho\}$

```python
make_qcs_gse_dsp_ancilla(hamiltonian: Hamiltonian = None,
                         sigma_list: List[Tuple[QuantumCircuit, int]] = None,
                         sigma_list_inv: List[Tuple[QuantumCircuit, int]] = None,
                         pauli_list: List[Tuple[Hamiltonian, int]] = None,
                         measurement: bool = True,
                         z_index: int = None, 
                         connection_graph: List[List[int]] = None,
                         barrier: bool = False) -> Tuple[List[QuantumCircuit], List[QuantumCircuit]]
```

### Define the decoder function of GSE
Given the raw measurement results of `hists_H_tilde` and `hists_S_tilde`, the function `compute_energy_gse_dsp_ancilla` computes the mitigated ground state energy by GSE.
Here we restrict the range of ground state energy according to the given Hamiltonian.
If we set `return_all == True`, then it returns all the $\tilde H$ and $\tilde S$ matrices in GSE, along with all the raw solutions of the generalized eigenvalue problem: $\tilde H\overrightarrow\alpha = E\tilde S\overrightarrow\alpha$.

```python
compute_energy_gse_dsp_ancilla(hists_H_tilde: List[Counts],
                               hists_S_tilde: List[Counts],
                               hamiltonian: Hamiltonian,
                               sigma_list: List[List[Tuple[QuantumCircuit, int]]],
                               pauli_list: List[Tuple[Hamiltonian, int]],
                               hist_type: str = "raw",
                               return_all: bool = False) -> Tuple[complex, np.array]
```

In [5]:
from gse import *
from dual_gse import *

# COMMON --- READ DATA (Main process starts from this cell.)

### Load the saved data
Here we load the VQE parameters from `pkls/parameters_vqe.pkl`.

We define 
- `num_qubits`:
- `num_layers`:
- `H`:
- `parameters_vqe`:

In [6]:
import pickle
with open("../pkls/parameters_vqe.pkl", "rb") as f:
    records_vqe = pickle.load(f)
num_qubits = records_vqe["num_qubits"]
num_layers = records_vqe["num_layers"]
H = records_vqe["H"]
parameters_vqe = records_vqe["record_param_list"][-1] # this is the log of parameters for each iteration. We use the last set of parameters.

### Define the backend and shot count
Here we define
- `backend`:
- `shots`:

In [7]:
from qiskit.providers.fake_provider import FakeJakarta
backend = FakeJakarta()
# backend = FakeJakarta(noise_model)
from qiskit import Aer
backend = Aer.get_backend("qasm_simulator")
shots = 1 << 13
initial_layout = [2, 0, 1, 3, 5]

### Define the function for generating the VQE circuit
This function can also be separated from this notebook as utility library.
- `add_vqe_layer(qc, qr, num_qubits, num_layers, param_list) -> None`

In [8]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
def add_vqe_layer(qc, qr, num_qubits, num_layers, param_list) -> None:
    for i in range(num_layers):
        for j in range(num_qubits): # RX gates and RZ gates
            qc.rx(param_list[i * num_qubits * 2 + j * 2], qr[j])
            qc.rz(param_list[i * num_qubits * 2 + j * 2 + 1], qr[j])
        for i in range(num_qubits)[0:-1:2]: # CZ gates
            qc.cz(qr[i], qr[i + 1])
        for i in range(num_qubits)[1:-1:2]: # CZ gates
            qc.cz(qr[i], qr[i + 1])
    for j in range(num_qubits): # RX gates and RZ gates
        qc.rx(param_list[num_layers * num_qubits * 2 + j * 2], qr[j])
        qc.rz(param_list[num_layers * num_qubits * 2 + j * 2 + 1], qr[j])

In [9]:
qr_vqe = QuantumRegister(num_qubits)
qc_vqe = QuantumCircuit(qr_vqe)
add_vqe_layer(qc_vqe, qr_vqe, num_qubits, num_layers, parameters_vqe)
# qc_vqe.draw("mpl")

### Create the circuits
- `qc_vqe`: the VQE circuit
- `qc_vqe_inv`: the inverse of the VQE circuit

In [10]:
from qiskit.compiler import transpile
basis_gates = ["rz", "sx", "cx"]
optimization_level = 3
qc_vqe = transpile(qc_vqe, basis_gates=basis_gates, optimization_level=optimization_level)
# qc_vqe.draw("mpl")
qc_vqe_inv = transpile(qc_vqe.inverse(), basis_gates=basis_gates, optimization_level=optimization_level)
# qc_vqe_inv.draw("mpl")

# Check the theoretical ground state energy

In [11]:
# set the records for later analysis
records_gse = dict()

In [12]:
eig_vals, eig_vecs = spy.linalg.eig(hamiltonian_to_dm(H))
energy_theoretical = sorted(eig_vals.real)[0]
### record the result ###
records_gse["theoretical"] = {"energy": energy_theoretical,
                              "eig_vals": eig_vals}
pprint(records_gse["theoretical"])

{'eig_vals': array([ 4.75877048+0.j, -4.75877048+0.j, -4.06417777+0.j,  2.75877048+0.j,
       -2.06417777+0.j, -1.69459271+0.j,  2.06417777+0.j, -0.30540729+0.j,
        0.30540729+0.j,  4.06417777+0.j, -2.75877048+0.j, -1.        +0.j,
        1.69459271+0.j,  1.        +0.j,  1.        +0.j, -1.        +0.j]),
 'energy': -4.758770483143625}


# Check the raw ground state energy

In [13]:
from qiskit.transpiler.passes import RemoveBarriers
from qiskit import execute, Aer
from qiskit.ignis.mitigation import expectation_value

qcs = []
for observable in H.keys():
    qr = QuantumRegister(num_qubits)
    cr = ClassicalRegister(num_qubits - observable.count("I"))
    qc = QuantumCircuit(qr, cr)
    qc.compose(qc_vqe, qubits=qr, inplace=True)
    add_observable(qc=qc, qr=qr, cr=cr, pauli_str=observable, inplace=True)
    # qc.save_density_matrix()
    qcs.append(qc)
qcs = transpile(qcs, basis_gates=basis_gates, optimization_level=optimization_level)

### run quantum circuits ###
job = execute(qcs, backend=backend, shots=shots, optimization_level=0)
# job = execute(qcs, backend=Aer.get_backend("qasm_simulator"), shots=1 << 17, optimization_level=0)
hists = job.result().get_counts()
# dms = get_density_matrices(job.result())
# dms_with_observables = [add_observable_to_dm(dm, observable) for dm, observable in zip(dms, H.keys())]
# hists = dms_to_hists(dms_with_observables)

### compute the expectation value of H ###
expval_raw = 0
for i, coeff in enumerate(H.values()):
    expval_raw += coeff * expectation_value(hists[i])[0]
    # expval_raw += coeff * dms_with_observables[i].trace().real
    
### record the result ###
records_gse["raw"] = {"eig_vals": None,
                      "energy": expval_raw}
print(records_gse["raw"])

{'eig_vals': None, 'energy': -4.77294921875}


# 
# Adding QREM before GSE
# 

[Quantum Readout Error Mitigation (QREM)](https://qiskit.org/textbook/ch-quantum-hardware/measurement-error-mitigation.html) mitigates the errors occured in measurement process.

In [14]:
from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit import QuantumRegister
qr_cal = QuantumRegister(num_qubits + 1)
meas_calibs, state_labels = complete_meas_cal(qr=qr_cal, circlabel='mcal')

In [15]:
from qiskit import execute
# Execute the calibration circuits
shots_cal = 1 << 13
job_cal = execute(meas_calibs, backend=backend, shots=shots_cal, optimization_level=0, initial_layout=initial_layout)

In [16]:
meas_fitter = CompleteMeasFitter(job_cal.result(), state_labels, circlabel='mcal')
meas_filter = meas_fitter.filter

# 
# GSE (by power subspace)
# 

# $\{\rho, \rho H\}$

### Make quantum circuits for power subspace

Here we set the subspace as $\{I, \rho H^k\}$.

$I, \rho, \rho H, \rho H^2$

$I, \rho, \rho^2, \rho H, \rho H^k, \rho^2 H, \rho^2 H^k$

In [14]:
qc_rho = qc_vqe
power_of_H = 2
print("power_of_H:", power_of_H)

power_of_H: 2


### specify the hardware restriction

In [15]:
z_index = 1
connection_order = [[0,1],[1,2],[2,3]]
print("z_index:", z_index)
print("connection_order:", connection_order)

z_index: 1
connection_order: [[0, 1], [1, 2], [2, 3]]


### Make quantum circuits
To use the real backend or check the results with finite shot count, we can only switch the attribute `measurement=False` to `measurement=True` in the function `make_qcs_gse_dsp_ancilla` here. (and also delete several unnecessary  cells for density matrix simulator.)

In [16]:
qcs_H_tilde, qcs_S_tilde = make_qcs_dgse_ps(H=H,
                                            qc_rho=qc_rho,
                                            power_of_H=power_of_H,
                                            measurement=True, # =False, 
                                            z_index=z_index, 
                                            connection_graph=connection_order, 
                                            barrier=False,
                                            flatten_qcs=True)
print("the number of quantum circuits in qcs_H_tilde:", len(qcs_H_tilde))
print("the number of quantum circuits in qcs_S_tilde:", len(qcs_H_tilde))

the number of quantum circuits in qcs_H_tilde: 279
the number of quantum circuits in qcs_S_tilde: 279


### Transpile the quantum circuit

In [17]:
### Transpile the quantum circuit# from qiskit.transpiler.passes import RemoveBarriers
qcs_H_tilde = [transpile(qc, basis_gates=basis_gates, optimization_level=0) for qc in qcs_H_tilde]
qcs_S_tilde = [transpile(qc, basis_gates=basis_gates, optimization_level=0) for qc in qcs_S_tilde]

### Run the quantum circuits

In [18]:
initial_layouts_H_tilde = []
for qc in qcs_H_tilde:
    if qc.num_qubits == num_qubits:
        initial_layouts_H_tilde.append(initial_layout[1:])
    else:
        initial_layouts_H_tilde.append(initial_layout)
initial_layouts_S_tilde = []
for qc in qcs_S_tilde:
    if qc.num_qubits == num_qubits:
        initial_layouts_S_tilde.append(initial_layout[1:])
    else:
        initial_layouts_S_tilde.append(initial_layout)

In [19]:
from qiskit import execute
job_qcs_H_tilde = execute(qcs_H_tilde, backend=backend, shots=shots, optimization_level=0, initial_layout=initial_layouts_H_tilde)
job_qcs_S_tilde = execute(qcs_S_tilde, backend=backend, shots=shots, optimization_level=0, initial_layout=initial_layouts_S_tilde)

### Retrieve jobs and make histograms

In [20]:
results_H_tilde = job_qcs_H_tilde.result()
results_S_tilde = job_qcs_S_tilde.result()
hists_H_tilde = results_H_tilde.get_counts()
hists_S_tilde = results_S_tilde.get_counts()

### Compute the mitigated expectation value
Here we output raw eigenvalues and eigenvectors of the generalize eigenvalue problem in GSE:
$$\mathcal{H} \vec{\alpha}=E \mathcal{S} \vec{\alpha}$$
After this, we remove the unphysical eigenvalues and choose the minimum one among the remainings.

In [21]:
eig_vals, eig_vecs, H_tilde, S_tilde = compute_energy_dgse_ps(hists_H_tilde=hists_H_tilde,
                                                              hists_S_tilde=hists_S_tilde,
                                                              H=H,
                                                              qc_rho=qc_rho,
                                                              power_of_H=power_of_H,
                                                              hist_type="raw",
                                                              return_all=True)
max_energy = len(H)
eig_vals_lower_bounded = eig_vals.real[- max_energy < eig_vals.real]
eig_vals_bounded = eig_vals_lower_bounded[eig_vals_lower_bounded < max_energy]
energy = np.sort(eig_vals_bounded)[0]
print("Estimated ground state energy by GSE (power subspace):", energy)
print()
print("physically meaningful eigen values")
pprint(eig_vals_bounded)
# print()
records_gse["power-subspace"] = {"eig_vals": eig_vals,
                              "eig_vecs": eig_vecs,
                              "H_tilde": H_tilde,
                              "S_tilde": S_tilde,
                              "energy": energy,}
# pprint(records_gse["qrem_power-subspace"])

Estimated ground state energy by GSE (power subspace): -4.967949275185369

physically meaningful eigen values
array([-4.96794928,  4.72007748,  1.87213773,  0.92295087])


### Apply QREM (if required)

In [24]:
# results_H_tilde_qrem = meas_filter.apply(connect_cregs(results_H_tilde))
# results_S_tilde_qrem = meas_filter.apply(connect_cregs(results_S_tilde))
# hists_H_tilde_qrem = results_H_tilde_qrem.get_counts()
# hists_S_tilde_qrem = results_S_tilde_qrem.get_counts()

In [25]:
eig_vals, eig_vecs, H_tilde, S_tilde = compute_energy_gse_dsp_ancilla(hists_H_tilde=hists_H_tilde_qrem,
                                                                      hists_S_tilde=hists_S_tilde_qrem,
                                                                      hamiltonian=H,
                                                                      sigma_list=sigma_list,
                                                                      pauli_list=pauli_list,
                                                                      hist_type="raw",
                                                                      return_all=True)
max_energy = len(H)
eig_vals_lower_bounded = eig_vals.real[- max_energy < eig_vals.real]
eig_vals_bounded = eig_vals_lower_bounded[eig_vals_lower_bounded < max_energy]
energy = np.sort(eig_vals_bounded)[0]
print("Estimated ground state energy by GSE (power subspace):", energy)
print()
print("physically meaningful eigen values")
pprint(eig_vals_bounded)
# print()
records_gse["qrem_power-subspace"] = {"eig_vals": eig_vals,
                              "eig_vecs": eig_vecs,
                              "H_tilde": H_tilde,
                              "S_tilde": S_tilde,
                              "energy": energy,}
# pprint(records_gse["qrem_power-subspace"])

NameError: name 'hists_H_tilde_qrem' is not defined

# Save Data

In [ ]:
import pickle
with open("pkls/results_gse_"+backend.name()+".pkl", "wb") as f:
    pickle.dump(records_gse, f)

# Qiskit versions

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table